In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import librosa
import librosa.display
import glob
import skimage
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def create_and_save_masks(clear_noise_path, background_noise_path, mask_output_path):
    # List files in the CV (clear noise) and BG (background noise) folders
    clear_noise_files = os.listdir(clear_noise_path)
    background_noise_files = os.listdir(background_noise_path)

    for clear_file in clear_noise_files:
        clear_name, _ = os.path.splitext(clear_file)

        for bg_file in background_noise_files:
            bg_name, _ = os.path.splitext(bg_file)
            cvpathtoload = os.path.join(clear_noise_path, clear_file)
            bgpathtoload = os.path.join(background_noise_path, bg_file)
            # Load clear and background noise audio files

            clear_noise, sr = librosa.load(f'{cvpathtoload}')
            background_noise, _ = librosa.load(f'{bgpathtoload}')
            clear_noise = clear_noise[:25000]
            background_noise = background_noise[:25000]

            # Compute the spectrograms
            stft_clear_noise = librosa.core.stft(clear_noise, n_fft=2048, hop_length=512)
            stft_background_noise = librosa.core.stft(background_noise, n_fft=2048, hop_length=512)
            bgn = np.abs(stft_background_noise)
            cn = np.abs(stft_clear_noise)

            # Calculate the mask using the formula
            mask = bgn / (cn + bgn + 1e-6)

            # Save the mask as a spectrogram in the MASK folder
            plt.figure(figsize=(10, 6))
            librosa.display.specshow(mask, sr=sr, hop_length=512)

            # Remove color bar
            plt.colorbar(format=None)

            # Remove title
            plt.title('')

            # Remove white borders
            plt.gca().xaxis.set_major_locator(plt.NullLocator())
            plt.gca().yaxis.set_major_locator(plt.NullLocator())

            # Save the mask spectrogram
            mask_file_name = f"{clear_name}_{bg_name}_mask.jpg"  # You can choose a different image format (e.g., .png, .jpg)
            plt.savefig(f'/content/drive/My Drive/SPECS/BGSPECS/{mask_file_name}', format='jpg', bbox_inches='tight', pad_inches=0)
            plt.close()

# Paths to the folders
clear_noise_path = '/content/drive/MyDrive/NCFOML/CV'
background_noise_path = '/content/drive/MyDrive/NCFOML/BG'
mask_output_path = '/content/drive/MyDrive/SPECS/MASKSPECS'

# Call the function to create and save masks for each type of background noise
create_and_save_masks(clear_noise_path, background_noise_path, mask_output_path)

In [ ]:
import os
import librosa
import numpy as np
import librosa.display
import matplotlib.pyplot as plt

def create_and_save_starget(clear_noise_path):
    # Load audio files
    background_noise_path = '/content/drive/MyDrive/NCFOML/BG/bgnoisecant.m4a'
    clear_noise, sr = librosa.load(clear_noise_path)
    background_noise, _ = librosa.load(background_noise_path)
    clear_noise = clear_noise[:25000]
    background_noise = background_noise[:25000]
    # combined_voice, _ = librosa.load(combined_voice_path)

    # Compute the spectrograms
    stft_clear_noise = np.abs(librosa.core.stft(clear_noise, n_fft=2048, hop_length=512))
    stft_background_noise = np.abs(librosa.core.stft(background_noise, n_fft=2048, hop_length=512))
    # stft_combined_voice = librosa.core.stft(combined_voice, n_fft=2048, hop_length=512)

    # Calculate the mask using the given formula
    mask = stft_background_noise / (stft_clear_noise + stft_background_noise+ 1e-06)

    # Calculate starget
    # starget = mask * stft_combined_voice

    # Convert starget to a log spectrogram
    log_spectogram = librosa.amplitude_to_db(mask)

    # Create a unique filename
    file_name = os.path.basename(clear_noise_path)  # Use the clear noise filename for naming the spectrogram

    # Plot and save the spectrogram of starget
    plt.figure(figsize=(10, 6))
    librosa.display.specshow(log_spectogram, sr=sr, hop_length=512)

    # Remove title
    plt.title('')

    # Remove white borders
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.gca().yaxis.set_major_locator(plt.NullLocator())
    output_path = '/content/drive/MyDrive/SPECS/MASKSPECS'

    # Save the image without borders, color bar, and title
    plt.savefig(os.path.join(output_path, f"{file_name}_masked_spectrogram_cant.jpg"), format='jpg', bbox_inches='tight', pad_inches=0)
    plt.close()

    return file_name

# Paths to the audio files and output folder
# clear_noise_path = '/path/to/clear_noise.wav'
# combined_voice_path = '/path/to/combined_voice.wav'
# output_path = '/path/to/output_folder'

# Call the function to create and save the spectrogram of starget


In [ ]:
import os

def process_files(source_folder_path):
    # List files in the source folder
    source_files = os.listdir(source_folder_path)

    for file_name in source_files:
        # Create the full file path
        file_path = os.path.join(source_folder_path, file_name)

        # Assuming you have clear noise and background noise folders
        # clear_noise_path = os.path.join(clear_noise_folder_path, file_name)  # Adjust as needed
        # background_noise_path = os.path.join(background_noise_folder_path, file_name)  # Adjust as needed

        # Call the function to create and save the spectrogram of the mask
        create_and_save_starget(file_path)

# Paths to the source folder and output folder
source_folder_path = '/content/drive/MyDrive/NCFOML/CV'
# output_folder_path = '/path/to/output_folder'

# Call the function to process files and create spectrogram images for masks
process_files(source_folder_path)

